In [1]:
import os
from openai import OpenAI

In [95]:
from IPython.display import display, HTML
from pprint import pprint

In [3]:
client = OpenAI()

In [4]:
# if you didn't set an environment variable, you can use set it manually here
# note that this is NOT good practice, and you shouldn't leave plaintext API keys in scripts!

import openai
openai.api_key  = ""

### Our completion function

In [46]:
def get_completion_better(messages, model="gpt-3.5-turbo", max_tokens = 1000, \
                          temperature = 1, top_p = 1, n = 1, stop = None, \
                          presence_penalty = 0, \
                          frequency_penalty = 0
                         ): 
    
    response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=messages,
          temperature=temperature,
          max_tokens=max_tokens,
          top_p=top_p,
          n = n,
        stop = stop,
        presence_penalty = presence_penalty,
        frequency_penalty = frequency_penalty
          
        )
    
    return response

#### Helper code to print out just the message content for multiple messages

In [41]:
def print_responses(response):
    for r in response.choices:
        print("\n", r.message.content)

#### Helper code for writing messages

In [42]:
prompt = ""
system_msg = ""

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

### Example: Prompting persona / persona prompting (you'll see either name)

You can shift a model's focus by asking the model to take on a particular persona. This can help align the output better to your intent. For example, give the model the persona of a math teacher for math questions, and an English teacher for grammar and writing tasks. 

In [188]:
prompt = "You are an English teacher skilled at fixing grammar in English sentences."
system_msg = "Fix the grammar of the following sentence: The sun roses in the morning."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

In [189]:
response = get_completion_better(messages)
print_responses(response)


 The correct sentence should be: The sun rose in the morning.


### Example: Asking for structured output

In [97]:
messages=[
    {
      "role": "system",
      "content": "You will be provided with unstructured data, and your task is to parse it into JSON format. Make sure \
      to include the actual content of the data in the output.",
         "type": "text"
    },
    {
      "role": "user",
      "content": "There are many fruits that were found on the recently discovered planet Goocrux. \
        There are neoskizzles that grow there, which are purple and taste like candy. \
        There are also loheckles, which are a grayish blue fruit and are very tart, \
        a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. \
        There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. \
        Finally, there are fruits called glowls, which have a very sour and bitter taste \
        which is acidic and caustic, and a pale orange tinge to them.",
         "type": "text"
    }
  ]

In [98]:
response = get_completion_better(messages)

In [99]:
print_responses(response)


 {
    "fruits_found_on_planet_Goocrux": {
        "neoskizzles": {
            "description": "Purple fruits that taste like candy",
            "color": "purple"
        },
        "loheckles": {
            "description": "Grayish blue fruits that are very tart, similar to lemons",
            "color": "grayish blue"
        },
        "pounits": {
            "description": "Bright green fruits that are more savory than sweet",
            "color": "bright green"
        },
        "loopnovas": {
            "description": "Neon pink fruits that taste like cotton candy",
            "color": "neon pink"
        },
        "glowls": {
            "description": "Fruits with a very sour, bitter, acidic, and caustic taste with a pale orange tinge",
            "color": "pale orange"
        }
    }
}


### Example: Adding prior chat history to API call

In [100]:
messages.append({"role": "assistant", "content": response.choices[0].message.content})
pprint(messages)

[{'content': 'You will be provided with unstructured data, and your task is to '
             'parse it into JSON format. Make sure       to include the actual '
             'content of the data in the output.',
  'role': 'system',
  'type': 'text'},
 {'content': 'There are many fruits that were found on the recently discovered '
             'planet Goocrux.         There are neoskizzles that grow there, '
             'which are purple and taste like candy.         There are also '
             'loheckles, which are a grayish blue fruit and are very '
             'tart,         a little bit like a lemon. Pounits are a bright '
             'green color and are more savory than sweet.         There are '
             'also plenty of loopnovas which are a neon pink flavor and taste '
             'like cotton candy.         Finally, there are fruits called '
             'glowls, which have a very sour and bitter taste         which is '
             'acidic and caustic, and a pale o

In [101]:
new_message = {
      "role": "user",
      "content": "A new fruit was discovered. It's named mloupmloup. It's hot pink in color and tastes like rainbows.",
         "type": "text"
    }

In [102]:
messages.append(new_message)

response = get_completion_better(messages)
print_responses(response)


 {
    "newly_discovered_fruit": {
        "name": "mloupmloup",
        "description": "Hot pink fruit that tastes like rainbows",
        "color": "hot pink"
    }
}


### Example: Helper function to add assistant response to current list of messsages

In [130]:
def add_assistant_response(messages, new_assistant_response):
    
    messages.append({"role": "assistant", "content": new_assistant_response.choices[0].message.content, "type":"text"})
    return messages

In [131]:
def add_new_user_messages(messages, new_user_prompt):
    messages.append({"role": "user", "content": new_user_prompt, "type":"text"})
    return messages

In [132]:
messages=[
    {
      "role": "system",
      "content": "You will be provided with unstructured data, and your task is to parse it into JSON format. Make sure \
      to include the actual content of the data in the output.",
         "type": "text"
    },
    {
      "role": "user",
      "content": "There are many fruits that were found on the recently discovered planet Goocrux. \
        There are neoskizzles that grow there, which are purple and taste like candy. \
        There are also loheckles, which are a grayish blue fruit and are very tart, \
        a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. \
        There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. \
        Finally, there are fruits called glowls, which have a very sour and bitter taste \
        which is acidic and caustic, and a pale orange tinge to them.",
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 {
    "fruits_found_on_planet_Goocrux": {
        "neoskizzles": {
            "description": "Purple fruits that taste like candy",
            "color": "purple",
            "taste": "sweet"
        },
        "loheckles": {
            "description": "Grayish blue fruits that are very tart, akin to lemon",
            "color": "grayish blue",
            "taste": "tart"
        },
        "pounits": {
            "description": "Bright green fruits that are more savory than sweet",
            "color": "bright green",
            "taste": "savory"
        },
        "loopnovas": {
            "description": "Neon pink fruits that taste like cotton candy",
            "color": "neon pink",
            "taste": "sweet"
        },
        "glowls": {
            "description": "Fruits with a sour, bitter, acidic, and caustic taste with a pale orange tinge",
            "color": "pale orange",
            "taste": "sour/bitter/acidity"
        }
    }
}


In [133]:
messages = add_assistant_response(messages, response)

new_user_prompt = f"A new fruit was discovered. It's called mloupmloup. It's dark green and tastes like grass."
messages = add_new_user_messages(messages, new_user_prompt)

In [134]:
response = get_completion_better(messages)
print_responses(response)


 {
    "newly_discovered_fruit": {
        "name": "mloupmloup",
        "description": "Dark green fruit that tastes like grass",
        "color": "dark green",
        "taste": "grassy"
    }
}


### Example: One-shot prompting

In [140]:
# Define the one-shot prompt
prompt = """
You are a creative writing assistant. Write a creative story beginning based on the prompt below:

Prompt: A dragon discovers a hidden village in the mountains.
Story: Once upon a time, in a faraway mountain range, a mighty dragon stumbled upon a village hidden among the peaks. The village was unlike any the dragon had ever seen...

Prompt: A young girl finds a magical amulet in her attic.
Story:
"""

# Make the API call with the one-shot prompt
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a creative writing assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message.content)

One radiant morning, a tenacious young girl named Lucy unearthed a secret in her own house. Amid the stacks of aged books, discarded furniture, and vintage trinkets in the dusty corners of her attic, something peculiar gracefully caught her attention. It was a unique amulet exuding hues of the morning sun, its aura enchanting like a magical artifact from an ancient lore.


### Example: Few-shot prompting

Does the story improve?

In [141]:
# Define the few-shot prompt
prompt = """
You are a creative writing assistant. Write a creative story beginning based on the prompt below:

Prompt: A dragon discovers a hidden village in the mountains.
Story: Once upon a time, in a faraway mountain range, a mighty dragon stumbled upon a village hidden among the peaks. The village was unlike any the dragon had ever seen...

Prompt: A young girl finds a magical amulet in her attic.
Story: Emily had always been curious about the old attic. One rainy afternoon, she decided to explore it and found a dusty, old box. Inside, there was a beautiful amulet that shimmered with a strange light...

Prompt: A spaceship lands in the middle of a desert.
Story: In the vast expanse of the desert, a bright light descended from the sky. As it touched the ground, the sand swirled around it, revealing a sleek spaceship. The hatch opened, and out stepped a figure...

Prompt: A detective receives a mysterious letter.
Story:
"""

# Make the API call with the few-shot prompt
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a creative writing assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message.content)

Detective Richard had been engrossed in case files when his assistant walked in holding a peculiar envelope. With a raised eyebrow, he opened the mysterious letter to find a coded message inside. This was the start of a case unlike any other he had ever solved...


### Example: Another example of few-shot prompting

In [166]:
prompt = f"""
Provide three synonyms for the given words:

Word: Happy
Synonyms: Joyful, Cheerful, Content

Word: Sad
Synonyms: Unhappy, Downcast, Miserable

Word: Angry
Synonyms:
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 Word: Angry
Synonyms: Furious, Irritated, Incensed


### Your task: Practice one-shot, two-shot and few-shot prompting

Pick a task for the model to do and try prompts that include no examples and prompts that include one or several examples. Do you see any improvement in the model outputs?

## Examples: Summarizing 

In [142]:
product_review = """

I recently purchased the pink stuffed animal panda, and I am absolutely thrilled with my purchase! The panda is the perfect size for cuddling, standing at about 12 inches tall. It’s not too big to be cumbersome, but large enough to make a significant presence on a bed or shelf.

The quality of this stuffed animal is top-notch. The fabric is incredibly soft to the touch, and the stitching is durable and well-done. I can tell this panda is made to last, even with regular use. The attention to detail, especially in the facial features and the cute little ears, is impressive.

This pink panda is beyond cute! The color is a lovely shade of pink, not too bright and very pleasing to the eyes. The panda has the sweetest expression, making it an instant favorite. It’s the perfect gift for anyone who loves pandas or simply enjoys collecting cute stuffed animals.

I was pleasantly surprised by how quickly the panda arrived. I placed my order on a Monday, and it was delivered by Wednesday, which is quite fast. The company clearly prioritizes prompt shipping. The packaging was excellent. The stuffed animal was placed in a plastic bag to protect it from any potential dirt or damage. This bag was then inside a sturdy box with ample padding to ensure that the panda arrived in pristine condition. I appreciate the care taken to pack the item securely.

Overall, I couldn’t be happier with this pink stuffed animal panda. It’s a perfect blend of size, quality, and cuteness, with the added bonus of fast shipping and secure packaging. Highly recommended for anyone looking for a delightful gift or a new addition to their stuffed animal collection!
"""

### Asking for a summary of a certain length

In [147]:
prompt = f"""
Please write a summary of a product review left on our ecommerce site.

The review is delimited by triple backticks. Summarize it in at most 30 words. 

Review: ```{product_review}```
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 This review raves about the pink stuffed animal panda, praising its quality, size, cuteness, and fast shipping. Highly recommended for those seeking a delightful gift.


### Summarize with a particular focus, e.g. quality or shipping

In [150]:
prompt = f"""
Our quality assurance department wants to know what customers think of the quality of our products.

Write a summary of a product review with a focus on anything related to quality.

The review is delimited by triple backticks. Summarize it in at most 30 words. 

Review: ```{product_review}```
"""

system_msg = "You are a helpful assistant who cares about the quality of products."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 High-quality pink stuffed panda with soft fabric, durable stitching, and impressive detailing. Fast shipping and secure packaging show care for the product. Recommended for gift or collection.


In [151]:
prompt = f"""
Our shipping partner wants to know what customers think of the shipping speed and packaging of our products.

Write a summary of a product review with a focus on anything related to shipping and how the product was packaged.

The review is delimited by triple backticks. Summarize it in at most 30 words. 

Review: ```{product_review}```
"""

system_msg = "You are a helpful assistant who cares about shipping."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 Fast shipping, secure packaging, high-quality product - customer thrilled with pink stuffed animal panda purchase. Highly recommended for size, quality, and cuteness.


## Your task: ask for summaries using different words, such as "extract" and "distill"

Try the above task again, but try to use different words instead of just summarize. You can ask ChatGPT to write you more fake reviews for products, or pull real product reviews for your favorite items!

### Example: Product review sentiment

In [152]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{product_review}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 The sentiment of the product review is overwhelmingly positive. The reviewer expresses great satisfaction with the pink stuffed animal panda, highlighting its size, quality, cuteness, fast shipping, and secure packaging. The language used throughout the review is enthusiastic and complimentary, indicating a very positive sentiment towards the product.


### ... the sentiment was correct, but that's a lot of words! Can we do better?

### Example: Asking for sentiment with one word

In [153]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" or "negative".

Review text: '''{product_review}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 positive


### Example: asking for types of emotions in a review

In [154]:
prompt = f"""
Identify a list of emotions that the writer of the following review is expressing. 

Include no more than five items in the list. 

Format your answer as a list of lower-case words separated by commas.

Review text: '''{product_review}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 excited, impressed, pleased, surprised, happy


### Example: Identify a specific type of emotion

In [155]:
prompt = f"""
Is the writer of the following review expressing anger?

The review is delimited with triple backticks. 

Give your answer as either yes or no.
Review text: '''{product_review}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 No


### Example: Identify multiple attributes from a review

In [156]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Color of the item

The review is delimited with triple backticks. 

Format your response as a JSON object with "Sentiment", "Anger", "Item" and "Color" as the keys.

If you don't know a value for a given attribute, write "NA".

Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{product_review}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 ```json
{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "stuffed animal panda",
    "Color": "pink"
}
```


### Example: Inferring topics from an article

In [157]:
# source: https://www.scientificamerican.com/article/carbon-removal-is-catching-on-but-it-needs-to-go-faster/

article = """
The world still isn’t sucking enough carbon dioxide out of the atmosphere to meet the Paris climate targets, scientists said Tuesday. And the gap grows wider every year that humanity delays meaningful cuts to global greenhouse gas emissions.

That’s the punchline of a new report on the state of global carbon dioxide removal, the practice of drawing CO2 out of the air to help tackle climate change. It's an update to the report's first edition, which was published in January 2023.

Nations worldwide are scrubbing about 2 billion metric tons of carbon dioxide each year, mainly by planting trees, the report says. But experts estimate they’ll need to remove at least 7 billion tons annually by midcentury.

Meanwhile, global emissions must fall rapidly to stay on the Paris track. Humanity spews nearly 40 billion metric tons of carbon dioxide each year through the burning of fossil fuels.

Scientists agree the primary strategy for tackling climate change is through a reduction of greenhouse gas emissions, mainly by phasing out fossil fuels and halting deforestation. But they also agree at least some carbon removal is necessary to keep global warming below 1.5 or 2 degrees Celsius, the major goals of the Paris Agreement.

That’s because global emissions must reach net zero within a few decades to meet the Paris timeline, meaning any remaining carbon going into the atmosphere must be counterbalanced by an equal amount coming out.

Carbon dioxide removal, or CDR, is the "only way really to provide a balance of net zero if we still have residual emissions in the system,” said Steve Smith, a climate science and policy expert at the University of Oxford and a lead author of the new report, at a press conference Tuesday.

The simplest way to hit net zero is to stop pouring carbon dioxide into the air. But some sectors of the economy likely cannot be fully decarbonized within the next few decades, either because the technology doesn’t exist yet or it can’t be scaled up quickly enough.

That means some residual emissions will be leftover by midcentury, and world leaders will need to offset them with carbon removal.

There are a variety of ways that can be done. Planting forests is the most popular strategy today, accounting for nearly all the carbon removal happening around the world. But researchers are working on a range of novel techniques on the side, from giant carbon-guzzling machines to special minerals that help the land or the ocean absorb more CO2.

Global interest and investment in carbon removal has risen in recent years, the report notes.

While novel CDR strategies account for less than 0.1 percent of global carbon removal capacity, they’re expanding faster than conventional methods, the new report finds. Grant funding for carbon removal research projects has steadily increased.

And there’s been a major jump in demonstration programs for new kinds of carbon removal techniques, particularly in the United States. The country’s first commercial direct air capture plant, built by Heirloom Carbon Technologies, opened in California last November.

Yet there’s still room for expansion. After a period of rapid growth in prior decades, new carbon removal patents have slowed since 2010. And while investment in carbon removal startups has generally increased over the past decade, it’s also declined after peaking in 2022.

There’s also a dearth of global policies that could incentivize companies to swiftly scale up their carbon removal capacity, the report notes.

For now, the voluntary carbon market — which allows companies and other carbon emitters to buy and sell carbon credits — is a small but growing source of demand for carbon removal projects.

Yet the market has attracted widespread criticism from experts who point out that carbon offsets are often less effective at reducing or removing emissions than the public is led to believe. And as of 2023, carbon removal credits accounted for less than 10 percent of the total credits sold on the voluntary carbon market.

That means there’s still a need for governments to implement policies that will spur more carbon removal innovation and expansion, the report suggests.

“We don’t see that policy signal yet,” said Greg Nemet, an environmental policy expert at the University of Wisconsin and another lead report author. “And we think that’s a really important lacking area that needs to be changed from a policy perspective.”

MIND THE GAP
Meanwhile, the report notes, countries must flesh out their long-term plans around emissions reductions and carbon removal. Recent studies have warned that most nations have not yet assembled comprehensive strategies for how they will achieve net zero in the coming decades.

Based on the long-term carbon removal plans that national governments have proposed, the new report estimates there’s still a significant gap between the amount of CDR expected by the year 2050 and the amount the world needs to keep temperatures below 1.5 degrees.

The size of the gap depends strongly on the strategies that world leaders use to reduce emissions and draw down carbon in the coming decades. The most sustainable pathways to meeting the Paris target generally suggest the world will need between 7 billion and 9 billion metric tons of carbon removal by the year 2050.

But it could be done with less. One of the most ambitious future scenarios that experts have modeled suggests the world could achieve 1.5C with only about 4.8 billion metric tons of annual carbon removal by the year 2050.

Based on countries’ current pledges, world leaders might get close. One of the best-case scenarios estimates the planet could be on track for about 4.4 billion tons of carbon removal by midcentury. That’s still a gap, but a relatively small one.

But that scenario comes with an important caveat. The analysis assumes that global greenhouse emissions are swiftly falling. They haven’t — in fact, they’re still rising.

That means the carbon removal gap likely is larger than the report suggests.

Even as experts say that global carbon removal is falling short, some scientists are worried more investment could backfire. They argue that a focus on carbon removal could detract from global efforts to reduce emissions — potentially lulling world leaders into the belief they can clean up their excess emissions with technology in the future.

But proponents of increased carbon removal say the practice is essential to achieving the Paris targets — and that world leaders should strive to reduce global emissions as quickly as possible.

“Meeting the Paris Agreement’s long-term temperature goals requires rapid greenhouse gas emissions reduction and near-term scale up of CDR,” said Smith, the University of Oxford scientist. “It’s not really an either-or situation.”

"""

In [158]:
prompt = f"""
Determine five topics that are being discussed in the following news story, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Article: '''{article}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 1. Carbon Dioxide Removal
2. Paris Climate Targets
3. Greenhouse Gas Emissions
4. Carbon Removal Technologies
5. Climate Change Mitigation


### Check whether certain topics are discussed in an article

In [159]:
list_of_topics = [ 'carbon dioxide', 
                 'paris climate accord',
                 'endangered species',
                 'chocolate',
                 'coffee']

In [161]:
prompt = f"""
Determine whether each item in the following list of topics is a topic in the text below, which is delimited with triple backticks.

Give your answer as a JSON with the name of the topic as the key and a 0 or 1 as the value. Use 0 if the topic is not in the article.

List of topics: {list_of_topics}

Article: '''{article}'''
"""

system_msg = "You are a helpful assistant."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 ```json
{
  "carbon dioxide": 1,
  "paris climate accord": 1,
  "endangered species": 0,
  "chocolate": 0,
  "coffee": 0
}
```


## Your task

Pick an article from your favorite publication and see if you can summarize it in different ways. Try to think of other ways of getting topics from the article. Some ideas:

* List of ideas with an indicator for whether the writer thinks the idea is positive or negative
* How important a particular idea is to the main point of the article
* Using more than or or two words per topic -- does asking for longer topics help or not? 

### Example: Chain of thought prompting

In [163]:
prompt = f"""
A high school student is solving the following problem:

A train travels from City A to City B at a speed of 60 miles per hour. \
On its return trip, the train travels at a speed of 40 miles per hour. \
If the total travel time for the round trip is 5 hours, what is the distance between City A and City B?

Let's break this problem down step by step (chain of thought) to find the solution. Give the solution if it exists.
"""

system_msg = "Use chain of thought when possible to give your answer."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 1. Let the distance between City A and City B be x miles. 

2. The time taken to travel from City A to City B at 60 mph is x/60 hours.

3. The time taken to travel from City B back to City A at 40 mph is x/40 hours.

4. The total travel time for the round trip is the sum of these two times: x/60 + x/40 = 5.

5. To solve for x, we can find a common denominator: 2x/120 + 3x/120 = 5.

6. Combining the fractions, we get 5x/120 = 5.

7. Dividing both sides by 5, we find x = 24.

Therefore, the distance between City A and City B is 24 miles.


## Your task

Try your hand at chain of thought prompting! You can ask ChatGPT for example math problems, for example. 

Play with the prompt to see if you can get better (or worse) steps. 

#### Bonus: Can you add one-shot prompting to help the model here?!

### Example: More chain of thought

In [165]:
prompt = f"""
Explain how photosynthesis works step-by-step.

For each step, summarize the step first with a couple of words. Write two or three sentences per step.
"""

system_msg = "Use chain of thought when possible to give your answer."

messages=[
    {
      "role": "system",
      "content": system_msg,
         "type": "text"
    },
    {
      "role": "user",
      "content": prompt,
         "type": "text"
    }
  ]

response = get_completion_better(messages)
print_responses(response)


 1. **Step 1 - Absorption of Light**: Plants absorb sunlight through pigments like chlorophyll located in their chloroplasts.
During this step, light energy is captured and converted into chemical energy in the form of ATP and NADPH.

2. **Step 2 - Water Splitting**: The absorbed light energy is used to split water molecules into oxygen, protons, and electrons.
This process, known as photolysis, releases oxygen as a byproduct into the atmosphere while providing electrons needed for the next stage.

3. **Step 3 - Carbon Fixation**: Carbon dioxide from the atmosphere is taken up by the plant through small openings called stomata.
Using the energy from ATP and NADPH, carbon fixation occurs in the Calvin cycle where carbon dioxide is converted into glucose (sugar) with the help of enzymes.

4. **Overall Process**: In summary, photosynthesis is a complex biochemical reaction where plants convert light energy into chemical energy to produce glucose, releasing oxygen as a byproduct. This pro

### Example: Asking for refinement of a response

In [168]:
initial_prompt = "Write two paragraphs about the importance of renewable energy technology."

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": initial_prompt}
    ]
)

initial_response = response.choices[0].message.content
print("Initial response:\n'", initial_response, "\n")

# Refine the response
refinement_prompt = f"Refine the following paragraphs to use less technical language:\n\n{initial_response}"

refined_response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": refinement_prompt}
    ]
)

print("Refined response:\n", refined_response.choices[0].message.content)

Initial response:
' Renewable energy technology is essential for the survival and progress of our planet. It addresses numerous environmental challenges such as global warming, ozone layer depletion, and acid rain, caused by the excessive utilization of fossil fuels. Renewable energy is a clean, inexhaustible, and increasingly competitive source of power that can substitute for the usage of fossil fuels in several applications. The use of renewable energy technology not only reduces the dependency on fossil fuel resources but also mitigates the harmful emissions and residues produced by conventional energy generation mechanisms. 

Moreover, the development and widespread implementation of renewable energy technologies can potentially boost economic growth and create jobs. The renewable energy sector is labor-intensive and can provide employment opportunities in manufacturing, installation, maintenance, and other related services. It also fosters innovation, technological progress and i

## Your task

Try your hand at asking for refinement of a given paragraph of text!

### Example: Additional context from an outside source

In [170]:
import requests

In [185]:
# Function to fetch weather information from OpenWeather API
def fetch_weather(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(base_url, params=params)
    return response.json()

# OpenWeather API key
openweather_api_key = ''

# Fetch weather information for a specific city
city = "Chicago"
weather_info = fetch_weather(city, openweather_api_key)

In [186]:
print(weather_info)

{'coord': {'lon': -87.65, 'lat': 41.85}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 10.86, 'feels_like': 9.93, 'temp_min': 7.04, 'temp_max': 12.24, 'pressure': 1017, 'humidity': 74}, 'visibility': 10000, 'wind': {'speed': 1.34, 'deg': 187, 'gust': 2.68}, 'clouds': {'all': 0}, 'dt': 1718100576, 'sys': {'type': 2, 'id': 2010190, 'country': 'US', 'sunrise': 1718100912, 'sunset': 1718155543}, 'timezone': -18000, 'id': 4887398, 'name': 'Chicago', 'cod': 200}


In [187]:
# Extract relevant details from the weather information
temperature = weather_info['main']['temp']
weather_description = weather_info['weather'][0]['description']

# Create a prompt using the fetched weather information
prompt = f"""
The current weather in {city} is {temperature}°C with {weather_description}. 

What kind of outfit do you suggest I wear today based on the weather.
"""

# Make the API call to OpenAI with the prompt
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message.content)

Based on the current weather, it might be a good idea to layer up. You can wear a light sweater or long sleeved shirt with a light to medium-weight jacket on top. Pair it with jeans or other warm pants and comfortable shoes. A scarf might also be useful if you feel a bit chilly. Don't forget to bring sunglasses since the sky is clear.


### Example: Using an outside API for context

In [181]:
# Function to fetch a random dog image from Dog CEO's Dog API
def fetch_random_dog_image():
    base_url = "https://dog.ceo/api/breeds/image/random"
    response = requests.get(base_url)
    return response.json()

# Fetch random dog image
dog_info = fetch_random_dog_image()

# Extract relevant details from the dog information
dog_image_url = dog_info['message']

# Create a prompt using the fetched dog image information
prompt = f"""
I just saw the cutest dog picture! Here's the image URL: {dog_image_url}. 

Can you write a short story about a day in the life of this dog?
"""

# Make the API call to OpenAI with the prompt
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message.content)

Once upon a time, in the corners of a bustling city, there lived an adorable Pomeranian named Pixie. Pixie was no ordinary Pomeranian; she was both bouncy and frolicsome, yet held a touch of elegance in her gleaming caramel fur. Her soulful eyes were as black as obsidian, containing a universe of curiosity and enthusiasm.

Even when the city was still asleep, her day would start with the break of dawn. The soft morning light illuminated her fur, setting it ablaze resembling a tiny, radiant sunrise. With a spring in her step and a wag in her tail, Pixie would embark on her daily morning adventures.

She was the only dog in her apartment complex, which made her quite popular among the residents. Her petite size didn't stop her from making friends with all the neighborhood dogs, big and small. From the sniffer on the first floor to the howler on the eighth, Pixie loved and was loved by all.

A typical morning included a playful chase with the squirrels in the local park. After enjoying th

In [182]:
print(dog_image_url)

https://images.dog.ceo/breeds/pomeranian/n02112018_3494.jpg


## Example: Batch prompting with k-shot context

In [190]:
# Define k-shot examples for sentiment analysis
k_shot_examples = """
Classify the sentiment of the following reviews as Positive, Negative, or Neutral.

Review: "I absolutely love this product! It exceeded all my expectations."
Sentiment: Positive

Review: "This is the worst purchase I have ever made. Completely dissatisfied."
Sentiment: Negative

Review: "The product is okay, not great but not terrible either."
Sentiment: Neutral
"""

# List of reviews to classify (batch input)
reviews = [
    "The service at the restaurant was fantastic, and the food was delicious.",
    "I didn't like the movie at all. It was too long and boring.",
    "The book was an interesting read, with both good and bad parts.",
    "The weather today is neither too hot nor too cold, just perfect."
]

# Function to create the prompt for each review
def create_prompt(review):
    return f"""
{k_shot_examples}

Review: "{review}"
Sentiment:
"""

# Collect responses for each review
responses = []

for review in reviews:
    prompt = create_prompt(review)
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    sentiment = response.choices[0].message.content
    responses.append((review, sentiment))

# Print the results
for review, sentiment in responses:
    print(f"Review: {review}\nSentiment: {sentiment}\n")

Review: The service at the restaurant was fantastic, and the food was delicious.
Sentiment: Positive

Review: I didn't like the movie at all. It was too long and boring.
Sentiment: Negative

Review: The book was an interesting read, with both good and bad parts.
Sentiment: Neutral

Review: The weather today is neither too hot nor too cold, just perfect.
Sentiment: Neutral



## Example: Prompt chaining

In [192]:
# Step 1: Extract key points from a piece of text

text = article # we'll use the article from the prior example 

extract_prompt = f"""
Extract the key points from the following text:

Text: "{text}"

Key Points:
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": extract_prompt}
    ]
)

key_points = response.choices[0].message.content
print("Extracted key points:\n", key_points, "\n\n")

# Step 2: Transform the key points into a bulleted list
transform_prompt = f"""
Transform the following key points into a bulleted list:

Key Points: "{key_points}"

Bulleted List:
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": transform_prompt}
    ]
)

bulleted_list = response.choices[0].message.content
print("Bulleted list:\n", bulleted_list, "\n\n")

# Step 3: Summarize the bulleted list into a concise paragraph
summarize_prompt = f"""
Summarize the following bulleted list into a concise paragraph:

Bulleted List: "{bulleted_list}"

Summary:
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": summarize_prompt}
    ]
)

summary = response.choices[0].message.content
print("Summary:\n", summary)

Extracted key points:
 - The world is not extracting enough carbon dioxide from the atmosphere to meet the Paris climate targets, and the gap is increasing every year, according to a new report.
- Currently, nations are removing around 2 billion metric tons of carbon dioxide a year by planting trees, but this needs to be at least 7 billion tons annually by mid-century.
- Scientists believe the primary strategy for addressing climate change should be a decrease in greenhouse gas emissions. Moreover, some carbon removal is necessary to limit global warming to 1.5 or 2 degrees Celsius.
- Some sectors of the economy will not be able to fully decarbonize in a few decades, meaning residual emissions will need to be offset by carbon removal.
- The most favored method of carbon removal is currently planting forests. However, new methods are in development, with a small number of these already contributing to global carbon removal capacity.
- Interest and investment in carbon removal have incre